In [ ]:
#Importand load the raw data
!unzip -o Dataset.zip

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 

fileNames=[]
text=[]

fileDir="Section2-Chapter2"
for fileName in os.listdir(fileDir):
    if fileName.endswith(".asm"):
        with open(fileDir+"/"+fileName, encoding='utf-8', errors='ignore') as f:
            content=f.read()
        content.replace(",", " ")
        content.replace('"', " ")
        text.append(content)
        
        fileName = fileName.split('.')[0]
        fileNames.append(fileName)


In [ ]:
#Extracting feature out of Text.

vectorizer=CountVectorizer(stop_words='english',max_features=1000)
dtm=vectorizer.fit_transform(text)
df=pd.DataFrame(dtm.toarray(),columns=vectorizer.get_feature_names())
df['file_name']=fileNames

df.head(10)

In [ ]:
#Feature Normalization
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer


reduced_svd=TruncatedSVD(2)
reduced_dtm=reduced_svd.fit_transform(dtm)
reduced_dtm=Normalizer(copy=False).fit_transform(reduced_dtm)

reduced_df=pd.DataFrame(reduced_svd.components_, index=["C1", "C2"],columns=vectorizer.get_feature_names())

reduced_df.head()


In [ ]:

final_df=pd.DataFrame(reduced_dtm,columns=["F1","F2"])
final_df.head()

In [ ]:
#Clustering and data labelling
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=3,random_state=0, batch_size=5, max_iter=10)
y_pred=kmeans.fit_predict(final_df)

final_df['label']=y_pred
final_df['file_name']=fileNames

final_df.head(20)


In [ ]:
final_df.groupby('label').count()

In [ ]:
#Data viualization 
from matplotlib import pyplot as plt

df1=final_df[final_df['label']==0]
df2=final_df[final_df['label']==1]
df3=final_df[final_df['label']==2]


plt.scatter(df1.F1, df1.F2, color='green')
plt.scatter(df2.F1, df2.F2, color='red')
plt.scatter(df3.F1, df3.F2, color='blue')

plt.xlabel('F1')
plt.ylabel('F2')


In [ ]:
#Saving the labelled data .
final_df.to_csv("Malware_Dataset_Labeled.csv.gz", index=False, compression="gzip")

In [ ]:
!ls -lh